# Creation du dataframe a partir du fichier csv

In [ ]:
import pandas as pd
df=pd.read_csv("airbnb_train.csv")

# Affichage des premieres lignes

Ici on remarque que les données ne sont pas en float ou int donc on ne pourra pas faire d'operation, il faut donc que les convertir en valeurs numérique.

In [ ]:
df.head(5)

# Affichage de statistiques

In [ ]:
df.describe()

# Netoyage de donnée

## Classe `Netoyage`
Cette classe permet de transformer le DataFrame en lui appliquant plusieurs methodes.

### Méthode `__init__`
- **Paramètre** :
  - `df` : Le DataFrame à transformer.
- **Action** : Initialise la classe avec le DataFrame.

### Méthode `transform`
- **Action** : Applique les différentes méthode de transformations au DataFrame.
- **Retour** : Retourne le DataFrame transformé.

### Méthode `convert_text`
- **Action** : Convertit les colonnes textuelles en valeurs numériques en remplaçant les valeurs uniques par des indices.
- **Colonnes concernées** : `neighbourhood`, `property_type`, `city`, `room_type`, `bed_type`, `cancellation_policy`.

### Méthode `convert_rate`
- **Action** : Convertit la colonne `host_response_rate` en pourcentage numérique et remplace les valeurs manquantes par 0.
- **Colonne concernée** : `host_response_rate`.

### Méthode `convert_bool`
- **Action** : Convertit les colonnes booléennes textuelles en valeurs numériques (1 pour `t` ou `True`, 0 pour autres).
- **Colonnes concernées** : `host_has_profile_pic`, `host_identity_verified`, `instant_bookable`, `cleaning_fee`.

### Méthode `convert_date`
- **Action** : Convertit les colonnes de date en nombre de jours écoulés depuis la date actuelle et remplace les valeurs manquantes par 0.
- **Colonnes concernées** : `first_review`, `last_review`, `host_since`.

### Méthode `replace_caracter`
- **Action** : Remplace les caractères spéciaux dans les noms de colonnes par des underscores.
- **Caractères remplacés** : `[^/{(&)}:-]+` et les espaces.

### Méthode `drop_unnecessary_columns`
- **Action** : Supprime les colonnes non utiles du DataFrame.
- **Colonnes supprimées** : `id`, `description`, `name`, `zipcode`.

### Méthode `handle_missing_values`
- **Action** : Remplit les valeurs manquantes par 0 dans le DataFrame.

### Méthode `create_amenities_columns`
- **Action** : Crée des colonnes binaires pour chaque commodité unique présente dans la colonne `amenities`.
- **Détail** :
  - Transforme la colonne `amenities` en une liste de commodités.
  - Crée une colonne pour chaque commodité unique avec des valeurs 1 ou 0 selon la présence de la commodité.
  - Supprime la colonne originale `amenities`.


In [ ]:
from datetime import datetime
import numpy as np

class Netoyage:
    def __init__(self, df):
        self.df = df
    
    def transform(self):
        self.create_amenities_columns()
        self.drop_unnecessary_columns()
        self.convert_text()
        self.convert_bool()
        self.convert_date()
        self.convert_rate()
        self.handle_missing_values()
        self.replace_caracter()
        return self.df
    
    def convert_text(self):
        tab = ['neighbourhood','property_type','city','room_type','bed_type','cancellation_policy']
        for col in tab:
            properties = self.df[col].unique()
            property2index = {prop:i for (i, prop) in enumerate(properties)}
            max_index = max(list(property2index.values()))
            self.df.loc[:, col] = self.df[col].replace(property2index)
            self.df.loc[self.df[col].map(type).eq(str),col] = np.nan
            self.df[col].fillna(max_index + 1)


    def convert_rate(self):
        self.df['host_response_rate']= self.df['host_response_rate'].str.rstrip('%').astype(float)
        self.df['host_response_rate']= self.df['host_response_rate'].fillna(0)
        
    def convert_bool(self):
        tab = ['host_has_profile_pic','host_identity_verified','instant_bookable','cleaning_fee']
        for col in tab:
            self.df[col] = self.df[col].apply(lambda x: 1 if (x == 't' or x == 'True') else 0)
            self.df[col] = self.df[col].fillna(2)


    def convert_date(self):
        tab = ['first_review', 'last_review', 'host_since']
        for col in tab:
            self.df[col] = pd.to_datetime(self.df[col])
            today = datetime.now()
            self.df[col] = (today - self.df[col]).dt.days
            self.df[col] = self.df[col].fillna(0)

    def replace_caracter(self):
        tab = '[^/{(&)}:-]+'
        for i in tab:
            self.df.columns = self.df.columns.str.replace(i, '_',)
        self.df.columns = self.df.columns.str.replace(' ', '_',)

    def drop_unnecessary_columns(self):
        columns_to_drop = ['id', 'description','name','zipcode']
        self.df = self.df.drop(columns=columns_to_drop)
    
    def handle_missing_values(self):
        self.df = self.df.fillna(0)
    
    def create_amenities_columns(self):
        self.df['amenities'] = self.df['amenities'].apply(lambda x: x.strip('{}').replace('"', '').split(','))
        unique_amenities = set(amenity.strip() for amenities_list in self.df['amenities'] for amenity in amenities_list)
        amenities_dict = {f'amenity_{amenity}': [] for amenity in unique_amenities}
        for amenities_list in self.df['amenities']:
            for amenity in unique_amenities:
                amenities_dict[f'amenity_{amenity}'].append(1 if amenity in amenities_list else 0)
        amenities_df = pd.DataFrame(amenities_dict)
        self.df = pd.concat([self.df, amenities_df], axis=1)
        self.df = self.df.drop(columns=['amenities'])

## Creation d'un objet de la classe Netoyage   
## Execute le netoyage des données     

In [ ]:
N = Netoyage(df)
df = N.transform()

## Affichage des premieres colonnes du dataframe transformé

On fait cela pour voir si tout est ok et que le netoyage a bien marché

In [ ]:
df.head(10)

## Affichage des colonnes 

In [ ]:
df.columns

# Affichage de la correlation

#### 1. Calculer la matrice de corrélation :  
J'utilise df.corr() pour calculer la matrice de corrélation pour toutes les colonnes de mon DataFrame df.

#### 2. Définir le seuil de corrélation :
Je définis un seuil de corrélation de 0.05 pour voir plus tard quelles colonnes ont un impact important.

#### 3. Filtrer les colonnes ayant une corrélation faible avec log_price :
J'identifie les colonnes dont la corrélation absolue avec log_price est supérieure au seuil défini. J'utilise abs(correlation_matrix["log_price"]) > threshold pour obtenir une liste de ces colonnes.

#### 4. Ajouter log_price à la liste des features corrélées :
Si log_price n'est pas déjà dans la liste des colonnes corrélées, je l'ajoute. Cela garantit que log_price est toujours inclus dans la matrice de corrélation filtrée.

#### 5. Créer une nouvelle matrice de corrélation avec les features filtrées :
Je crée une nouvelle matrice de corrélation en utilisant uniquement les colonnes sélectionnées, ce qui permet de se concentrer sur les corrélations pertinentes

#### 6. Extraire uniquement la colonne log_price des corrélations filtrées :
J'extrais la colonne log_price de cette nouvelle matrice de corrélation pour obtenir uniquement les corrélations des autres variables avec log_price.

#### 7. Visualiser les corrélations de log_price avec un heatmap :
J'utilise seaborn pour créer une heatmap qui visualise les corrélations entre log_price et les autres variables sélectionnées. J'ajoute des annotations pour afficher les valeurs exactes des corrélations et j'utilise une palette de couleurs coolwarm.

#### 8. Afficher les colonnes corrélées dans le DataFrame :
J'affiche les colonnes sélectionnées du DataFrame en utilisant df[correlated_features]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Calculer la matrice de corrélation
correlation_matrix = df.corr()

# Définir le seuil de corrélation
threshold = 0.15

# Filtrer les colonnes ayant une corrélation faible avec log_price
correlated_features = correlation_matrix.index[abs(correlation_matrix["log_price"]) > threshold].tolist()

# Ajouter log_price à la liste des features corrélées si ce n'est pas déjà fait
if 'log_price' not in correlated_features:
    correlated_features.append('log_price')

# Créer une nouvelle matrice de corrélation avec les features filtrées
filtered_corr_matrix = df[correlated_features].corr()

# Extraire uniquement la colonne 'log_price' des corrélations filtrées
log_price_corr = filtered_corr_matrix[['log_price']]

# Visualiser les corrélations avec un heatmap
plt.figure(figsize=(12, 12))
sns.heatmap(filtered_corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5, cbar=False)
plt.title('Correlation with log_price')
plt.show()

# tableau de correlation

1. J'affiche la correlation de chaque colonne avec log_price   
2. J'affiche le dataframe avec les colonnes qui ont une grande correlation avec le log_price

In [ ]:
# Visualiser les corrélations de log_price avec un heatmap
plt.figure(figsize=(8, 12))
sns.heatmap(filtered_corr_matrix[['log_price']], annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5, cbar=False)
plt.title('Correlation with log_price')
plt.show()

df[correlated_features]

# Affichage de la distribution des valeurs des colonnes

1. Log Price:
Le graphique montre une distribution bimodale des prix (log-transformés), avec des pics principaux autour de 4 et 5. Cela suggère deux groupes principaux de prix de location.

2. Room Type:
La majorité des Airbnb sont du type "0", suivie par "1". Le type "2" est nettement moins fréquent. (Les types de chambre sont probablement codés numériquement).

3. Accommodates:
La plupart des propriétés peuvent accueillir entre 1 et 4 personnes, avec une forte concentration sur ceux qui accueillent 2 personnes.

4. Bathrooms:
La majorité des propriétés ont 1 salle de bain, et le nombre diminue nettement pour 2 salles de bains et plus.

5. Bedrooms:
La majorité des propriétés a 1 ou 2 chambres à coucher.

6. Beds:
La distribution montre que la plupart des propriétés ont entre 1 et 3 lits.

7. Amenities (TV, Cable TV, Indoor fireplace, etc.):
Ces histogrammes montrent la fréquence des différentes commodités. Par exemple, beaucoup de propriétés ont une TV, moins ont le câble, encore moins ont une cheminée intérieure.
Les commodités comme les lave-linges, sèche-linges et les éléments adaptés aux familles sont également représentés, chacun avec des fréquences variées.

In [ ]:
import matplotlib.pyplot as plt

dfv=df[correlated_features]
numerical_cols = dfv.select_dtypes(include=['int', 'float']).columns
fig, axes = plt.subplots(len(numerical_cols), 1, figsize=(4, 4 * len(numerical_cols)))
for i, col in enumerate(numerical_cols):
    dfv[col].hist(ax=axes[i])
    axes[i].set_title(f'Distribution of {col}')
plt.tight_layout()

# Affichage d'une carte avec des couleurs des airbnb en fonction du log price

In [ ]:
# Créer une carte centrée autour des moyennes des latitudes et longitudes
import folium

data = df
map = folium.Map(location=[data['latitude'].mean(), data['longitude'].mean()], zoom_start=10)

def color_map(p):
    if p < 4:
        return "blue"
    if 4 < p < 5:
        return "orange"
    else:
        return "red"


# Ajouter des points sur la carte pour chaque Airbnb
for idx, row in data.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=5,
        popup=f"Price: {row['log_price']}\\nRoom: {row['room_type']}\\nAccommodates: {row['accommodates']}",
        color=color_map(row['log_price']),
        fill=True,
        fill_color=color_map(row['log_price'])
    ).add_to(map)

# Afficher la carte
map

# import des librairies

j'importe les librairies que je vais utilisé pour créer mes modeles

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

# Creation des Modeles de ML

- Séparer les features (X) et la target (y) :  
Je commence par séparer les caractéristiques (X) de la variable cible (y) en supprimant la colonne log_price de mon DataFrame pour obtenir X. Ensuite, j'assigne log_price à y.

- Diviser les données en ensembles d'entraînement et de test :     
Je divise mes données en ensembles d'entraînement et de test en utilisant train_test_split avec 80 % des données pour l'entraînement et 20 % pour le test. J'utilise random_state pour assurer la reproductibilité.

- Initialiser les paramètres :     
J'initialise les différents paramètres pour initialiser mes modèles

- Entrainer le modèle :
J'utilise la fonction fit (sauf pour la descente de gradient) pour entrainer mon model sur les 80% de données d'entrainements.

- Faire des prédictions :      
J'utilise la fonction predict (sauf pour la descente de gradient) pour faire des prédictions sur les ensembles d'entraînement (X_train) et de test (X_test).

- Évaluer le modèle :      
J'évalue les performances du modèle en calculant le coefficient de détermination (R²) pour les ensembles d'entraînement et de test en utilisant r2_score.

- J'affiche les scores R² pour les ensembles d'entraînement et de test.

# Modele Descente de Gradient Classique

In [ ]:
# Séparer les features (X) et la target (y)
X = df.drop(columns=['log_price'])
y = df['log_price']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardiser les données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialiser les paramètres
n_samples, n_features = X_train_scaled.shape
learning_rate = 0.1
n_iterations = 1000

# Initialiser les poids et le biais
weights = np.zeros(n_features)
bias = 0

# Descente de gradient batch
for i in range(n_iterations):
    # Calcul des prédictions
    y_pred = np.dot(X_train_scaled, weights) + bias
    
    # Calcul des gradients
    dw = (1 / n_samples) * np.dot(X_train_scaled.T, (y_pred - y_train))
    db = (1 / n_samples) * np.sum(y_pred - y_train)
    
    # Mise à jour des poids et du biais
    weights -= learning_rate * dw
    bias -= learning_rate * db

# Faire des prédictions
y_pred_train = np.dot(X_train_scaled, weights) + bias
y_pred_test = np.dot(X_test_scaled, weights) + bias

# Évaluer le modèle
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)
print(f'R-squared (train): {r2_train}')
print(f'R-squared (test): {r2_test}')


# Modele Regression Lineaire

In [ ]:
# Séparer les features (X) et la target (y)
X = df.drop(columns=['log_price'])
y = df['log_price']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardiser les données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Créer le modèle de régression linéaire
model = LinearRegression()

# Entraîner le modèle
model.fit(X_train_scaled, y_train)

# Faire des prédictions
y_pred_test = model.predict(X_test_scaled)
y_pred_train = model.predict(X_train_scaled)

# Évaluer le modèle
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)
print(f'R-squared (train): {r2_train}')
print(f'R-squared (test): {r2_test}')

# Model RandomForestRegressor

In [ ]:
# Séparer les features (X) et la target (y)
X = df.drop(columns=['log_price'])
y = df['log_price']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Créer le modèle de régression par forêts aléatoires
rfr_model = RandomForestRegressor(n_estimators=50, max_depth=20, random_state=42)

# Entraîner le modèle
rfr_model.fit(X_train, y_train)

# Faire des prédictions
y_pred_test = rfr_model.predict(X_test)
y_pred_train = rfr_model.predict(X_train)

# Évaluer le modèle
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)
print(f'R-squared (train): {r2_train}')
print(f'R-squared (test): {r2_test}')


# Model GradientBoostingRegressor

In [ ]:
# Séparer les features (X) et la target (y)
X = df.drop(columns=['log_price'])
y = df['log_price']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Créer le modèle de régression par gradient boosting
gb_model = GradientBoostingRegressor(n_estimators=50, learning_rate=0.5, max_depth=5, random_state=42)

# Entraîner le modèle
gb_model.fit(X_train, y_train)

# Faire des prédictions
y_pred_test = gb_model.predict(X_test)
y_pred_train = gb_model.predict(X_train)

# Évaluer le modèle
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)
print(f'R-squared (train): {r2_train}')
print(f'R-squared (test): {r2_test}')


# Model XGBoost

In [ ]:
# Séparer les features (X) et la target (y)
X = df.drop(columns=['log_price'])
y = df['log_price']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

# Creation du model XGRegressor 
xgb_model = XGBRegressor(n_estimators=1000, learning_rate=0.05, max_depth=6, subsample=1, colsample_bytree=1, random_state=20)

# Entraîner le modèle
xgb_model.fit(X_train, y_train)

# Faire des prédictions
y_pred_test = xgb_model.predict(X_test)
y_pred_train= xgb_model.predict(X_train)

# Évaluer le modèle
r2_test = r2_score(y_test, y_pred_test)
r2_train = r2_score(y_train, y_pred_train)
print(f'R-squared (train): {r2_train}')
print(f'R-squared (test): {r2_test}')

# Test que j'ai effectué

- J'ai testé de standardiser les données pour voir si cela avait une influence sur mon r2 score mais j'ai remarquer que cela n'avait aucune influence pour tout mes modeles.
- J'ai aussi essayé d'appliquer le PCA mais cela a diminuer mon r2 score sur mon train et test.
- J'ai testé différents parametres pour les modeles pour essayer d'améliorer le r2_score de mes modeles.

# interpretation des résultats

On remarque que le meilleur modele que j'ai codé est le XGBoost car j'obtient un r2_score de 0.71 sur le test et 0.90 sur le train.
Cela veut dire que 71% des données du test est expliquée par le modele.

# Compare les predictions et les vrais prix
Sauvegarde les vraies valeurs et les valeurs prédites dans un fichier csv

In [ ]:
airbnb_test = pd.read_csv('airbnb_test.csv')
N=Netoyage(airbnb_test)
df_test=N.transform()
df_train=df.drop('log_price', axis=1)

# supprimer les colonnes qui ne sont pas dans df_train
for col in df_test.columns:
    if col not in df_train.columns:
        df_test = df_test.drop(columns=col)

# aligner les colonnes de df_train et df_test
df_test_aligned = df_test.reindex(columns=df_train.columns)

# fait les predictions avec mon meilleur modèle (le XGBoost)
pred = xgb_model.predict(df_test_aligned)

# sauvegarde les predictions
compare = pd.DataFrame({'':airbnb_test['id'],'logpred': pred})
compare.to_csv('predictions.csv', index=False)
print('Les resultats ont été sauvgarder')

# Test de la conformité de mon fichier "predictions.csv"

In [ ]:
def estConforme(monFichier_csv):
    votre_prediction = pd.read_csv(monFichier_csv)

    fichier_exemple = pd.read_csv("prediction_example.csv")

    assert votre_prediction.columns[1] == fichier_exemple.columns[1], f"Attention, votre colonne de prédiction doit s'appeler {fichier_exemple.columns[1]}, elle s'appelle '{votre_prediction.columns[1]}'"
    assert len(votre_prediction) == len(fichier_exemple), f"Attention, vous devriez avoir {len(fichier_exemple)} prédiction dans votre fichier, il en contient '{len(votre_prediction)}'"

    assert np.all(votre_prediction.iloc[:,0] == fichier_exemple.iloc[:, 0])

    print("Fichier conforme!")

estConforme("predictions.csv")